<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>


In [185]:
import pandas as pd

In [186]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [187]:
df.drop(
    ["PassengerId", "Name", "SibSp", "Parch", "Ticket", "Cabin", "Embarked"],
    axis="columns",
    inplace=True,
)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [188]:
inputs = df.drop("Survived", axis="columns")
target = df.Survived

In [189]:
# inputs.Sex = inputs.Sex.map({"male": 1, "female": 2})
# inputs

In [190]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,False,True
1,True,False
2,True,False


In [191]:
inputs = pd.concat([inputs, dummies], axis="columns")
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,False,True
1,1,female,38.0,71.2833,True,False
2,3,female,26.0,7.9250,True,False


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**


In [192]:
inputs.drop(["Sex", "male"], axis="columns", inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True


In [193]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [194]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [195]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.Age[:6]

,Pclass,Fare,female
Age,,,
0.42,3.0,8.5167,0.0
0.67,2.0,14.5000,0.0
0.75,3.0,19.2583,1.0
0.83,2.0,23.8750,0.0
0.92,1.0,151.5500,0.0
...,...,...,...
70.00,1.5,40.7500,0.0
70.50,3.0,7.7500,0.0
71.00,1.0,42.0792,0.0


In [196]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.3)

In [197]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

In [198]:
model.fit(X_train, y_train)

GaussianNB()

In [199]:
model.score(X_test, y_test)

0.7574626865671642

In [200]:
X_test[0:10]

,Pclass,Age,Fare,female
361,2,29.000000,27.7208,False
836,3,21.000000,8.6625,False
731,3,11.000000,18.7875,False
629,3,29.699118,7.7333,False
258,1,35.000000,512.3292,True
548,3,33.000000,20.5250,False
314,2,43.000000,26.2500,False
650,3,29.699118,7.8958,False
675,3,18.000000,7.7750,False
62,1,45.000000,83.4750,False


In [201]:
y_test[0:10]

361    0
836    0
731    0
629    0
258    1
548    0
314    0
650    0
675    0
62     0
Name: Survived, dtype: int64

In [202]:
model.predict(X_test[0:10])

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

In [203]:
model.predict_proba(X_test[:10])

array([[9.28452302e-01, 7.15476976e-02],
       [9.61470195e-01, 3.85298050e-02],
       [9.56173249e-01, 4.38267508e-02],
       [9.65217972e-01, 3.47820277e-02],
       [5.06936872e-51, 1.00000000e+00],
       [9.66849534e-01, 3.31504657e-02],
       [9.38829762e-01, 6.11702382e-02],
       [9.65254274e-01, 3.47457260e-02],
       [9.59667436e-01, 4.03325642e-02],
       [3.81702900e-01, 6.18297100e-01]])

**Calculate the score using cross validation**


In [204]:
from sklearn.model_selection import cross_val_score

cross_val_score(GaussianNB(), X_train, y_train, cv=5)

array([0.76      , 0.824     , 0.816     , 0.76612903, 0.75806452])